In [1]:
import pandas as pd

In [2]:
# 取出所有消费总额超过 50 元的用户的所有订单号order_id
# 读取数据集
df1 = pd.read_excel('./示例文件.xlsx',sheet_name='data1')
df1

,Order_id（主键）,City_id,User_id,Merchandise,Quantity,Amount
0,1,10001,15442,土鸡蛋4枚,7,28
1,2,10001,15442,早餐面包,12,24
2,3,10001,22009,土鸡蛋4枚,40,160
3,4,10002,39502,土鸡蛋4枚,4,16
4,5,10002,39502,早餐面包,11,22


In [3]:
# 求出符合要求的用户清单
df2 = df1.groupby(['User_id'])['Amount'].sum().reset_index()
df2 = df2[df2['Amount'] > 50]['User_id']
df2

0    15442
1    22009
Name: User_id, dtype: int64

In [4]:
# 合并表
df_merge = pd.merge(df1,df2,on='User_id',how='inner')
df_merge

,Order_id（主键）,City_id,User_id,Merchandise,Quantity,Amount
0,1,10001,15442,土鸡蛋4枚,7,28
1,2,10001,15442,早餐面包,12,24
2,3,10001,22009,土鸡蛋4枚,40,160


In [5]:
# 求出符合要求的订单号
order_df = df_merge['Order_id（主键）'].drop_duplicates()
order_df

0    1
1    2
2    3
Name: Order_id（主键）, dtype: int64

In [6]:
# 每个城市售卖商品分普通团购和秒杀两种方式，各自均有购买时间限制，秒杀都会放在一个普通团购时间区间内
# 将所有订单（Order_id）所属的普通团购时间（Groupon_date）查询出来（秒杀时间对应到普通团购日期）
# 读取数据集
df1 = pd.read_excel('./示例文件.xlsx',sheet_name='data2_1')
df1

,Group_id（主键）,City_id,Groupon_date,Group_type,Start_time,End_time
0,1,10001,2020-11-11,团购,2020-11-11 20:00:00,2020-11-12 19:58:00
1,2,10001,2020-11-12,秒杀,2020-11-12 10:00:00,2020-11-12 10:59:59
2,3,10001,2020-11-12,秒杀,2020-11-12 14:00:00,2020-11-12 14:59:59
3,4,10002,2020-11-11,团购,2020-11-11 21:00:00,2020-11-12 20:59:00
4,5,10002,2020-11-11,秒杀,2020-11-11 22:00:00,2020-11-12 00:00:00


In [7]:
df2 = pd.read_excel('./示例文件.xlsx',sheet_name='data2_2')
df2

,Order_id（主键）,City_id,Group_id,User_id,Merchandise
0,NaN,NaN,（外键）,NaN,NaN
1,1.0,10001.0,2,15442.0,土鸡蛋4枚
2,2.0,10001.0,1,15442.0,早餐面包
3,3.0,10001.0,3,22009.0,土鸡蛋4枚
4,4.0,10002.0,5,39502.0,土鸡蛋4枚
5,5.0,10002.0,4,39502.0,早餐面包


In [8]:
# 合并表
df_merge = pd.merge(df2,df1,left_on='Group_id',right_on='Group_id（主键）',how='inner')
df_merge

,Order_id（主键）,City_id_x,Group_id,User_id,Merchandise,Group_id（主键）,City_id_y,Groupon_date,Group_type,Start_time,End_time
0,1.0,10001.0,2,15442.0,土鸡蛋4枚,2,10001,2020-11-12,秒杀,2020-11-12 10:00:00,2020-11-12 10:59:59
1,2.0,10001.0,1,15442.0,早餐面包,1,10001,2020-11-11,团购,2020-11-11 20:00:00,2020-11-12 19:58:00
2,3.0,10001.0,3,22009.0,土鸡蛋4枚,3,10001,2020-11-12,秒杀,2020-11-12 14:00:00,2020-11-12 14:59:59
3,4.0,10002.0,5,39502.0,土鸡蛋4枚,5,10002,2020-11-11,秒杀,2020-11-11 22:00:00,2020-11-12 00:00:00
4,5.0,10002.0,4,39502.0,早餐面包,4,10002,2020-11-11,团购,2020-11-11 21:00:00,2020-11-12 20:59:00


In [9]:
# 条件筛选
df = df_merge[['Order_id（主键）','Groupon_date']]
df['Order_id（主键）'] = df['Order_id（主键）'].astype(int)
df

D:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Order_id（主键）,Groupon_date
0,1,2020-11-12
1,2,2020-11-11
2,3,2020-11-12
3,4,2020-11-11
4,5,2020-11-11


In [10]:
# 每个城市设置团期开始结束时间区间不会重叠
# 查询出每个城市（City_id）每个团期（Groupon_date）上次团期结束时间和本次团期结束时间
# 读取数据集
df1 = pd.read_excel('./示例文件.xlsx',sheet_name='data3')
df1

,Id,City_id,Groupon_date,Start_time,End_time
0,1,10001,2020-11-11,2020-11-11 20:00:00,2020-11-12 19:58:00
1,2,10002,2020-11-11,2020-11-11 21:00:00,2020-11-12 20:59:00
2,3,10001,2020-11-12,2020-11-12 20:00:00,2020-11-13 19:58:00
3,4,10002,2020-11-12,2020-11-12 21:00:00,2020-11-13 20:59:00
4,5,10001,2020-11-14,2020-11-14 20:00:00,2020-11-15 19:58:00


In [11]:
# 排序
df1 = df1.sort_values(by=['City_id','Groupon_date','Start_time'],ascending=True)
df1

,Id,City_id,Groupon_date,Start_time,End_time
0,1,10001,2020-11-11,2020-11-11 20:00:00,2020-11-12 19:58:00
2,3,10001,2020-11-12,2020-11-12 20:00:00,2020-11-13 19:58:00
4,5,10001,2020-11-14,2020-11-14 20:00:00,2020-11-15 19:58:00
1,2,10002,2020-11-11,2020-11-11 21:00:00,2020-11-12 20:59:00
3,4,10002,2020-11-12,2020-11-12 21:00:00,2020-11-13 20:59:00


In [12]:
# 组内移动
df1['last_End_time'] = df1.groupby(['City_id'])['End_time'].shift(1)
df1

,Id,City_id,Groupon_date,Start_time,End_time,last_End_time
0,1,10001,2020-11-11,2020-11-11 20:00:00,2020-11-12 19:58:00,NaT
2,3,10001,2020-11-12,2020-11-12 20:00:00,2020-11-13 19:58:00,2020-11-12 19:58:00
4,5,10001,2020-11-14,2020-11-14 20:00:00,2020-11-15 19:58:00,2020-11-13 19:58:00
1,2,10002,2020-11-11,2020-11-11 21:00:00,2020-11-12 20:59:00,NaT
3,4,10002,2020-11-12,2020-11-12 21:00:00,2020-11-13 20:59:00,2020-11-12 20:59:00


In [13]:
# 选择列
df = df1[['City_id','Groupon_date','last_End_time','End_time']]
df = df.fillna('')
df

,City_id,Groupon_date,last_End_time,End_time
0,10001,2020-11-11,,2020-11-12 19:58:00
2,10001,2020-11-12,2020-11-12 19:58:00,2020-11-13 19:58:00
4,10001,2020-11-14,2020-11-13 19:58:00,2020-11-15 19:58:00
1,10002,2020-11-11,,2020-11-12 20:59:00
3,10002,2020-11-12,2020-11-12 20:59:00,2020-11-13 20:59:00
